##### Нахождение закономерностей в данных:

На бумаге были исследованы случаи с записями model_id==35231362.

##### Этапы решения:

|Номер | Этап | Val_score | Pub_score |
|-------|:---:|:-------------|:-----------|
|   1   | Старт | 0.627 | NA |
|   2   | Small tuning | 0.651 |
|   3   | Добавлен признак missing | 0.832      |  0.818    |
|   4   | missing float       | 0.853      |  0.847    | 
|   5   | Добавлен признак count   | 0.858     |   0.852    |
|   6   | Добавлен признак max_pageview_duration   | 0.871     |   0.866  |
|   NA  | Первый бейслайн побит!   | NA     |  NA     |
|   7   | Добавлен признак diff_last_time   | 0.92536     |   0.931    |
|   8   | Добавлен признак совпадающего model_id   |  Failed   |  NA   |
|   9   | Добавлен признак min_time без учета model_id   |  Failed   | NA   |
|   10   | Добавлен признак min_time   |  0.933   |  0.937   |
|   NA   |  Второй бейслайн побит!  |  NA   |  NA   |
|   11  | join c webstat по session_id and time_exact_before   | 0.941     |     0.942  |
|   12   | join с webstat по session_id and time_exact_before2   | 0.9726     |   0.970    |
|   13   |  join с webstat по session_id and time_exact_after  | 0.9728     |     NA  |
|   14   |  Отбор признаков  |  Failed   |  NA   |
|   15   |  Добавлен признак diff_time  |  0.974   |  0.975   |


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

import lightgbm as lgb
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

##### До первого бейслайна:

In [2]:
df = pd.read_csv("train.csv")

test_df = pd.read_csv("test.csv")

In [3]:
df.shape[0]+test_df.shape[0]

121791

In [4]:
X = df.drop("is_callcenter",axis=1)
y = df["is_callcenter"]

X = pd.concat([X, test_df], ignore_index=True)

In [5]:
webstat = pd.read_csv("t1_webstat.csv")

webstat = webstat.sort_values("date_time")

In [6]:
df

,order_id,create_time,good_id,price,utm_medium,utm_source,sessionkey_id,category_id,parent_id,root_id,model_id,is_moderated,rating_value,rating_count,description_length,goods_qty,pics_qty,model_create_time,is_callcenter
0,1269921,1975-12-26 09:30:08,9896348,753,5,8.0,123777004,139,133,124,123517,1,5.0,6.0,1204,6,2,1971-04-14 00:15:20.000,1
1,1270034,1975-12-26 10:28:57,9896348,753,1,2.0,123781654,139,133,124,123517,1,5.0,6.0,1204,6,2,1971-04-14 00:15:20.000,0
2,1268272,1975-12-25 11:24:28,9896348,753,2,3.0,123591002,139,133,124,123517,1,5.0,6.0,1204,6,2,1971-04-14 00:15:20.000,1
3,1270544,1975-12-26 14:16:06,9896348,753,1,1.0,123832302,139,133,124,123517,1,5.0,6.0,1204,6,2,1971-04-14 00:15:20.000,1
4,1270970,1975-12-26 18:21:47,9896348,753,3,56.0,123881603,139,133,124,123517,1,5.0,6.0,1204,6,2,1971-04-14 00:15:20.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104590,1250981,1975-12-16 12:11:32,68657110,1497,4,1.0,121795335,2873,1229,1183,35231362,1,4.0,3.0,1183,12,1,1976-01-23 00:08:20.000,1
104591,1173775,1975-11-08 11:29:50,60391507,1354,1,2.0,114277105,2873,1229,1183,35231362,1,4.0,3.0,1183,12,1,1976-01-23 00:08:20.000,1
104592,1180920,1975-11-12 09:14:35,66971400,1452,2,NaN,114972131,2873,1229,1183,35231362,1,4.0,3.0,1183,12,1,1976-01-23 00:08:20.000,0
104593,1177882,1975-11-10 17:28:07,60391498,1433,4,2.0,114672813,2873,1229,1183,35231362,1,4.0,3.0,1183,12,1,1976-01-23 00:08:20.000,1


In [7]:
webstat

,sessionkey_id,date_time,page_type,pageview_number,pageview_duration_sec,category_id,model_id,good_id,price,product_in_sale
2268917,109996122,1975-10-17 13:42:56.953,2,1,11.0,722.0,NaN,NaN,NaN,NaN
2268918,109996122,1975-10-17 13:43:07.510,2,2,22.0,7196.0,NaN,NaN,NaN,NaN
2268919,109996122,1975-10-17 13:43:29.860,2,3,25.0,779.0,NaN,NaN,NaN,NaN
2269206,109996122,1975-10-17 13:43:54.757,2,4,9.0,7196.0,NaN,NaN,NaN,NaN
2267445,109996122,1975-10-17 13:44:03.803,2,5,11.0,723.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2267023,134622417,1976-02-16 22:55:29.327,3,120,26.0,NaN,NaN,NaN,NaN,NaN
2267024,134622417,1976-02-16 22:55:55.497,1,121,17.0,4271.0,31165696.0,NaN,NaN,NaN
2078414,134622417,1976-02-16 22:56:12.727,2,122,159.0,4271.0,NaN,NaN,NaN,NaN
2000326,134622417,1976-02-16 22:58:51.717,7,123,51.0,NaN,NaN,NaN,NaN,NaN


Количество записей(count) и пропущенных записей(missing) в `webstat` по `sessionkey_id`

In [8]:
pages = webstat.groupby("sessionkey_id")["pageview_number"].agg(['max','count'])

pages['missing'] = pages['max'] - pages['count']

pages = pages.drop(['max'],axis=1)

X = X.join(pages, 'sessionkey_id', how='left')

Максимальная длительность пребывания на одной странице(max)

In [9]:
dur = webstat.groupby("sessionkey_id")["pageview_duration_sec"].agg(['max'])

X = X.join(dur, 'sessionkey_id', how='left')

Прошедшее время между самой последней записью в `webstat` и записью в датасете по `sessionkey_id` (diff_last_min)

In [10]:
last_time = webstat.groupby("sessionkey_id")["date_time"].agg(['last'])
X = X.join(last_time, 'sessionkey_id', how='left')


X["last"] = X["last"].astype("datetime64")
X["create_time"] = X["create_time"].astype("datetime64")


X["diff_last_min"]=X["create_time"]-X["last"]
X["diff_last_min"] = X["diff_last_min"].apply(lambda x: x.total_seconds()/60)


##### До второго бейслайна:

Прошедшее время между последней записью в `webstat` до записи в датасете по `sessionkey_id`,`good_id` (min_time_sec)

In [11]:
webstat["date_time"] = webstat["date_time"].astype("datetime64")

date_list = webstat.groupby(["sessionkey_id", "good_id"])["date_time"].apply(list)

X = X.join(date_list, ["sessionkey_id","good_id"], how='left')


def min_time(create_time,date_time):
    if type(date_time)==float:
        return np.NaN
    else:
        min_time = datetime.timedelta(days=1000).total_seconds()
        for times in date_time:
            if (times<create_time) and ((create_time-times).total_seconds()<min_time):
                min_time = (create_time-times).total_seconds()
        if (min_time == datetime.timedelta(days=1000).total_seconds()):
            min_time = np.NaN
        return min_time



X['min_time_sec'] = X.apply(lambda x: min_time(x.create_time,x.date_time), axis=1)

#### После второго бейслайна 

Join датасета с `webstat` по последней записи по времени до записи в датасете с совпадающими `sessionkey_id`,`good_id` 

In [12]:
def min_time_exact(create_time,date_time):
    if type(date_time)==float:
        return np.NaN
    else:
        min_date = datetime.timedelta(days=1000).total_seconds()
        res_time = np.NaN
        for times in date_time:
            if (times<create_time) and ((create_time-times).total_seconds()<min_date):
                min_date = (create_time-times).total_seconds()
                res_time = times
        return res_time
    
    
X['time_exact_before'] = X.apply(lambda x: min_time_exact(x.create_time,x.date_time), axis=1)

In [13]:
X = pd.merge(X,webstat[["sessionkey_id","date_time","page_type","pageview_number","pageview_duration_sec"]], left_on=["sessionkey_id","time_exact_before"], right_on=["sessionkey_id","date_time"], how='left')
X = X.drop_duplicates(subset="order_id")

Аналогичный join, но без учёта `good_id` 

In [14]:
date_list = webstat.groupby(["sessionkey_id"])["date_time"].apply(list)

X = X.join(date_list, ["sessionkey_id"], how='left')

X['min_time_sec2'] = X.apply(lambda x: min_time(x.create_time,x.date_time), axis=1)    
X['time_exact_before2'] = X.apply(lambda x: min_time_exact(x.create_time,x.date_time), axis=1)

In [15]:
X = pd.merge(X,webstat[["sessionkey_id","date_time","page_type","pageview_number","pageview_duration_sec"]], left_on=["sessionkey_id","time_exact_before2"], right_on=["sessionkey_id","date_time"], how='left',suffixes=('_a','_b'))
X = X.drop_duplicates(subset='order_id')

Количество пропущенных записей в `webstat` в момент появления записи в датасете. (page_diff)

In [16]:
def time_after(create_time,date_time):
    if type(date_time)==float:
        return np.NaN
    else:
        min_time = datetime.timedelta(days=1000).total_seconds()
        for times in date_time:
            if (times>create_time) and ((times-create_time).total_seconds()<min_time):
                min_time = (times-create_time).total_seconds()             
        if (min_time == datetime.timedelta(days=1000).total_seconds()):
            min_time = np.NaN
        return min_time

def time_exact_after(create_time,date_time):
    if type(date_time)==float:
        return np.NaN
    else:
        min_time = datetime.timedelta(days=1000).total_seconds()
        res_time = np.NaN
        for times in date_time:
            if (times>create_time) and ((times-create_time).total_seconds()<min_time):
                min_time = (times-create_time).total_seconds()
                res_time = times
        return res_time

In [17]:
X['time_after'] = X.apply(lambda x: time_after(x.create_time,x.date_time_a), axis=1)
X['time_exact_after'] = X.apply(lambda x: time_exact_after(x.create_time,x.date_time_a), axis=1)

In [18]:
X = pd.merge(X, webstat[["sessionkey_id","date_time","page_type","pageview_number","pageview_duration_sec"]], left_on=["sessionkey_id","time_exact_after"], right_on=["sessionkey_id","date_time"], how='left',)
X = X.drop_duplicates(subset='order_id')

In [19]:
col = ['order_id', 'price', 'utm_source', 'root_id', 'rating_value', 'count', 'missing', 'max', 'diff_last_min', 'min_time_sec', 'pageview_number_a', 'pageview_duration_sec_a', 'min_time_sec2', 'page_type_b', 'pageview_number_b', 'pageview_duration_sec_b',"time_after","pageview_number"]
X = X[col]
X = X.set_index("order_id")

In [20]:
X["page_diff"] = X["pageview_number"] - X["pageview_number_b"]

In [21]:
test = X[104595:]
X = X[:104595]

In [22]:
X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=True, test_size=0.3, random_state=40)

In [ ]:
gbm = lgb.LGBMClassifier(learning_rate=0.05, n_estimators=370, first_metric_only = True)

gbm.fit(X_train, y_train ,eval_set =[(X_val,y_val)] , eval_metric=['auc'],
        early_stopping_rounds=500,verbose = 2)

In [24]:
y_pred = gbm.predict_proba(X_val)

roc_auc_score(y_val, y_pred[:,1])

0.9738037008858272

In [25]:
y_tr = gbm.predict_proba(X_train)

roc_auc_score(y_train, y_tr[:, 1])

0.9844236222797644

In [ ]:
clf = CatBoostClassifier(
    iterations=700,
    random_seed=42,
    learning_rate=0.05,
    custom_loss=['AUC']
)

clf.fit(
    X, y,
    verbose=False)

res2 = clf.predict_proba(test)

In [ ]:
gbm = lgb.LGBMClassifier(learning_rate=0.05, n_estimators=370, first_metric_only = True)
gbm.fit(X, y, verbose = 2)

res1 = gbm.predict_proba(test)

In [28]:
res = (res1[:,1] + res2[:,1])/2

In [29]:
data = {'order_id': test.index,
    'is_callcenter': res}

subm = pd.DataFrame(data)


subm.to_csv('submission12.csv', index=False)